In [ ]:
!pip install sentencepiece

### 普通模型删减head

In [1]:
!python3 example.py

original_attention_layer:MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
)
剪枝后的多头注意力层已保存！


### vila模型删减head

In [19]:
!ls ../../base_model/VILA1.5-3b

config.json  llm  mm_projector	README.md  trainer_state.json  vision_tower


##### 观察模型，构造删减head json

In [ ]:
import json
from transformers import AutoModel

# Load the VILA 1.5-3B model
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "../../base_model/VILA1.5-3b/llm"
model = AutoModel.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)

keys = []
for name, module in model.named_modules():
    print(name, type(module))

    # LLama里是transformers.models.llama.modeling_llama.LlamaSdpaAttention，不是nn.MultiheadAttention
    # if isinstance(module, nn.MultiheadAttention):
    if 'self_attn' in name:
        if name.split('.')[-1]=='self_attn':
            keys.append("model."+name) # 注意这里命名有区别
            print(f"{name} module.num_heads: {module.num_heads}")
            
print(keys)
content = {}
for k in keys:
    # content[f'model.layers.{i}.self_attn'] = [0,1,2]
    content[k] = [0,1,2,3]

with open("prune_dict_vila.json",'w') as output_file:
    output_file.write(json.dumps(content, ensure_ascii=False))

##### 删减head

In [ ]:
# !python3 example_vila.py --pretrained_model '../../base_model/VILA1.5-3b/llm' --prune_dict 'prune_dict_vila.json' --output_path 'outputs/vila-llm-pruned-mask'

In [2]:
!python3 example_vila_mask.py --pretrained_model '../../base_model/VILA1.5-3b/llm' --prune_dict 'prune_dict_vila.json' --output_path 'outputs/vila-llm-pruned-mask'

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00,  3.15it/s]
layer model.layers.0.self_attn has 20 heads
Pruning heads [0, 1, 2, 3] in layer model.layers.0.self_attn
layer model.layers.1.self_attn has 20 heads
Pruning heads [0, 1, 2, 3] in layer model.layers.1.self_attn
layer model.layers.2.self_attn has 20 heads
Pruning heads [0, 1, 2, 3] in layer model.layers.2.self_attn
layer model.layers.3.self_attn has 20 heads
Pruning heads [0, 1, 2, 3] in layer model.layers.3.self_attn
layer model.layers.4.self_attn has 20 heads
Pruning heads [0, 1, 2, 3] in layer model.layers.4.self_attn
layer model.layers.5.self_attn has 20 heads
Pruning heads [0, 1, 2, 3] in layer model.layers.5.self_attn
layer model.layers.6.self_attn has 20 heads
Pruning heads [0, 1, 2, 3] in layer model.layers.6.self_attn
layer model.layers.7.self_attn has 20 heads
Pruning heads [0, 1, 2, 3] in layer model.layers.7.self_attn
layer model.layers.8.self_attn has 20 heads
Pruning heads [0, 1, 2, 3] in layer

### llama3-8B模型删减head

In [ ]:
# 用llama3模型做实验，假设删减掉每个attention层的前3个head

In [1]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "../../base_model/Meta-Llama-3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)

keys = []
for name, module in model.named_modules():
    # print(name, type(module))

    # LLama里是transformers.models.llama.modeling_llama.LlamaSdpaAttention，不是nn.MultiheadAttention
    # if isinstance(module, nn.MultiheadAttention):
    if 'self_attn' in name:
        if name.split('.')[-1]=='self_attn':
            keys.append(name)
            print(f"{name} module.num_heads: {module.num_heads}")
            
print(keys)
content = {}
for k in keys:
    # content[f'model.layers.{i}.self_attn'] = [0,1,2]
    content[k] = [0,1,2]

with open("prune_dict_llama3.json",'w') as output_file:
    output_file.write(json.dumps(content, ensure_ascii=False))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

model.layers.0.self_attn module.num_heads: 32
model.layers.1.self_attn module.num_heads: 32
model.layers.2.self_attn module.num_heads: 32
model.layers.3.self_attn module.num_heads: 32
model.layers.4.self_attn module.num_heads: 32
model.layers.5.self_attn module.num_heads: 32
model.layers.6.self_attn module.num_heads: 32
model.layers.7.self_attn module.num_heads: 32
model.layers.8.self_attn module.num_heads: 32
model.layers.9.self_attn module.num_heads: 32
model.layers.10.self_attn module.num_heads: 32
model.layers.11.self_attn module.num_heads: 32
model.layers.12.self_attn module.num_heads: 32
model.layers.13.self_attn module.num_heads: 32
model.layers.14.self_attn module.num_heads: 32
model.layers.15.self_attn module.num_heads: 32
model.layers.16.self_attn module.num_heads: 32
model.layers.17.self_attn module.num_heads: 32
model.layers.18.self_attn module.num_heads: 32
model.layers.19.self_attn module.num_heads: 32
model.layers.20.self_attn module.num_heads: 32
model.layers.21.self_at

In [ ]:
# llama3-8B model比较特殊，https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py
# 它使用了group attention，以至于它的query num_heads 和k,v num_heads 是不一样多的
# self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=config.attention_bias)
# self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads
# 这里的逻辑需要单独处理

In [17]:
!python3 example_llama.py --pretrained_model '../../base_model/Meta-Llama-3-8B-Instruct' --prune_dict 'prune_dict_llama3.json' --output_path 'outputs/llama3-8b-instruct-pruned'

Loading checkpoint shards: 100%|██████████████████| 4/4 [00:00<00:00, 13.73it/s]
layer model.layers.0.self_attn has 32 heads
Pruning heads [0, 1, 2] in layer model.layers.0.self_attn
LlamaSdpaAttention(
  (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
  (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
  (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): LlamaRotaryEmbedding()
)
layer model.layers.0.self_attn remains 29 heads
layer model.layers.1.self_attn has 32 heads
Pruning heads [0, 1, 2] in layer model.layers.1.self_attn
LlamaSdpaAttention(
  (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
  (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
  (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): LlamaRotaryEmbedding()
)
l

In [3]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "outputs/llama3-8b-instruct-pruned"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)

keys = []
for name, module in model.named_modules():
    # print(name, type(module))

    # LLama里是transformers.models.llama.modeling_llama.LlamaSdpaAttention，不是nn.MultiheadAttention
    # if isinstance(module, nn.MultiheadAttention):
    if 'self_attn' in name:
        if name.split('.')[-1]=='self_attn':
            keys.append(name)
            print(f"{name} module.num_heads: {module.num_heads}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

model.layers.0.self_attn module.num_heads: 32
model.layers.1.self_attn module.num_heads: 32
model.layers.2.self_attn module.num_heads: 32
model.layers.3.self_attn module.num_heads: 32
model.layers.4.self_attn module.num_heads: 32
model.layers.5.self_attn module.num_heads: 32
model.layers.6.self_attn module.num_heads: 32
model.layers.7.self_attn module.num_heads: 32
model.layers.8.self_attn module.num_heads: 32
model.layers.9.self_attn module.num_heads: 32
model.layers.10.self_attn module.num_heads: 32
model.layers.11.self_attn module.num_heads: 32
model.layers.12.self_attn module.num_heads: 32
model.layers.13.self_attn module.num_heads: 32
model.layers.14.self_attn module.num_heads: 32
model.layers.15.self_attn module.num_heads: 32
model.layers.16.self_attn module.num_heads: 32
model.layers.17.self_attn module.num_heads: 32
model.layers.18.self_attn module.num_heads: 32
model.layers.19.self_attn module.num_heads: 32
model.layers.20.self_attn module.num_heads: 32
model.layers.21.self_at

### 训练删减后的模型

In [55]:
!cp -r ../../base_model/VILA1.5-3b outputs

In [4]:
!cp -r outputs/vila-llm-pruned-mask outputs/VILA1.5-3b/llm

In [ ]:
!bash VILA-catastrophic-forgetting-exp-main/scripts/v1_5/release/8b/3_sft_anker.sh outputs/VILA1.5-3b outputs/VILA1.5-3b-trained